In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Note

* The joined_XX.csv is left joined in the SQL with code:
* SELECT * FROM df_XX
> LEFT JOIN X_initial
> ON df_XX.gameId = X_initial.gameId and df_XX.playId = X_initial.playId and df_XX.kickerId = X_initial.nflId
* Play description is droped for punt and field+extra before join, since there will be error when import into SQL sever
* The punt and filed+extra joined file have some redundant index, and currently all features not appropriate for the training is manually dropped (can improve further)
* Another improvement might on standardize which feature
* From the result, only kickoff and field+extra, which with only 2 mainly play result, have high accuracy. The punt play type with more play result 



In [ ]:
joined_df = pd.read_csv(input())

# Drop records doesn't has data matched from tracking dataset
joined_df = joined_df.dropna(subset=['x']).reset_index()
joined_df.info()

/content/drive/MyDrive/NEU/Kaggle/joined_FE.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5981 entries, 0 to 5980
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   level_0                 5981 non-null   int64  
 1   Column_1                5981 non-null   int64  
 2   gameId                  5981 non-null   int64  
 3   playId                  5981 non-null   int64  
 4   quarter                 5981 non-null   int64  
 5   down                    5981 non-null   int64  
 6   yardsToGo               5981 non-null   int64  
 7   possessionTeam          5981 non-null   object 
 8   specialTeamsPlayType    5981 non-null   object 
 9   specialTeamsResult      5981 non-null   object 
 10  kickerId                5981 non-null   float64
 11  returnerId              1 non-null      float64
 12  kickBlockerId           9 non-null      float64
 13  yardlineSide            5981 non-null   objec

In [ ]:
joined_df.head(3)

,level_0,Column_1,gameId,playId,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,returnerId,kickBlockerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,Column_1.1,index,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId.1,playId.1,playDirection
0,0,2,2018090600,658,1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,NaN,NaN,PHI,3,5:03:00,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13,69.0,12073850.0,2018-09-07 01:31:25.9000000,23.47,21.00,0.11,0.71,0.01,316.41,208.40,None,27091.0,Matt Bryant,3.0,K,away,1.0,2.018091e+09,658.0,left
1,1,6,2018090600,1368,2,4,8,PHI,Field Goal,Kick Attempt Good,44966.0,NaN,NaN,ATL,8,6:12:00,NaN,NaN,NaN,0,3,NaN,26.0,NaN,0,18,115.0,12083441.0,2018-09-07 01:59:53.5000000,28.72,21.42,0.07,0.06,0.01,292.12,259.59,None,44966.0,Jake Elliott,4.0,K,home,1.0,2.018091e+09,1368.0,left
2,2,8,2018090600,1587,2,4,17,ATL,Field Goal,Kick Attempt Good,27091.0,NaN,NaN,PHI,34,2:13:00,NaN,NaN,NaN,3,3,NaN,52.0,NaN,0,76,228.0,12085166.0,2018-09-07 02:11:41.3000000,65.05,28.38,0.02,0.03,0.00,128.34,106.17,None,27091.0,Matt Bryant,3.0,K,away,1.0,2.018091e+09,1587.0,right


In [ ]:
# Data is inbalanced, mainly Return and Touchback
joined_df.groupby('specialTeamsResult').size()

specialTeamsResult
Blocked Kick Attempt       9
Kick Attempt Good       5361
Kick Attempt No Good     611
dtype: int64

# Kickoff

* Prepare data for fitting



In [ ]:
# Drop records with too many null
ml_kick = joined_df.drop(['gameId','playId','index','Column_1','playDescription','returnerId','penaltyCodes','penaltyJerseyNumbers','penaltyYards','kickReturnYardage','gameClock'], axis=1)
# Fill with '' since the dtype of column is object
ml_kick['yardlineSide'] = ml_kick['yardlineSide'].fillna('')
# Transform time format and convert to total seconds
ml_kick['time'] = pd.to_datetime(ml_kick['time'], format = '%M:%S.%f').dt.time
idx = pd.to_timedelta(ml_kick['time'].astype(str))
ml_kick['time'] = idx.dt.total_seconds()
ml_kick.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14058 entries, 0 to 14057
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   quarter                 14058 non-null  int64  
 1   down                    14058 non-null  int64  
 2   yardsToGo               14058 non-null  int64  
 3   possessionTeam          14058 non-null  object 
 4   specialTeamsPlayType    14058 non-null  object 
 5   specialTeamsResult      14058 non-null  object 
 6   kickerId                14058 non-null  int64  
 7   yardlineSide            14058 non-null  object 
 8   yardlineNumber          14058 non-null  int64  
 9   preSnapHomeScore        14058 non-null  int64  
 10  preSnapVisitorScore     14058 non-null  int64  
 11  kickLength              14058 non-null  int64  
 12  playResult              14058 non-null  int64  
 13  absoluteYardlineNumber  14058 non-null  int64  
 14  time                    14058 non-null

* Classification

In [ ]:
# Use label encoder to transform categorical data
subset = ml_kick.loc[:,['possessionTeam','specialTeamsPlayType','specialTeamsResult','yardlineSide','event','team','playDirection']]
subset = subset.apply(LabelEncoder().fit_transform)
subset.head(3)

,possessionTeam,specialTeamsPlayType,specialTeamsResult,yardlineSide,event,team,playDirection
0,25,0,4,26,0,1,1
1,25,0,4,26,0,1,0
2,25,0,4,26,0,1,0


In [ ]:
# Standardize numerical data

ml_kick = ml_kick.drop(['possessionTeam','specialTeamsPlayType','specialTeamsResult','yardlineSide','event','team','playDirection'], axis=1)
norm = StandardScaler()
ml_kick = pd.DataFrame(norm.fit_transform(ml_kick),columns = ml_kick.columns)
ml_kick

,quarter,down,yardsToGo,kickerId,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,kickLength,playResult,absoluteYardlineNumber,time,x,y
0,-1.231288,0.0,0.0,1.204000,-0.02376,-1.131762,-1.126086,-0.184874,0.276620,-0.930742,-1.332500,-1.545722,0.488535
1,-0.385439,0.0,0.0,1.204000,-0.02376,-0.832936,-0.742471,0.560453,0.276620,1.056808,-1.486171,1.564269,-0.517023
2,0.460409,0.0,0.0,1.204000,-0.02376,-0.135674,-0.358856,0.933117,-0.788063,1.056808,-0.546976,1.549752,-0.509288
3,1.306258,0.0,0.0,1.204000,-0.02376,0.661196,0.408374,0.684675,0.276620,-0.930742,-0.824510,-1.575564,0.296705
4,-1.231288,0.0,0.0,0.992978,-0.02376,-1.131762,-1.126086,-0.557537,0.631514,-0.930742,-1.509463,-1.625569,0.493176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14053,1.306258,0.0,0.0,1.242349,-0.02376,0.561587,3.093680,-0.557537,-0.362190,-0.930742,-0.450450,0.845648,1.682828
14054,1.306258,0.0,0.0,1.242349,-0.02376,0.561587,3.093680,-0.557537,-0.362190,-0.930742,-0.450352,0.845648,1.689016
14055,1.306258,0.0,0.0,1.242349,-0.02376,0.561587,3.093680,-0.557537,-0.362190,-0.930742,-0.450253,0.845648,1.696751
14056,1.306258,0.0,0.0,1.242349,-0.02376,0.561587,3.093680,-0.557537,-0.362190,-0.930742,-0.450154,0.845648,1.706033


In [ ]:
# Concat numerical and encoded categorical data
ml_kick = pd.concat([ml_kick, subset], axis = 1)
ml_kick.head(3)

,quarter,down,yardsToGo,kickerId,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,kickLength,playResult,absoluteYardlineNumber,time,x,y,possessionTeam,specialTeamsPlayType,specialTeamsResult,yardlineSide,event,team,playDirection
0,-1.231288,0.0,0.0,1.204,-0.02376,-1.131762,-1.126086,-0.184874,0.276620,-0.930742,-1.332500,-1.545722,0.488535,25,0,4,26,0,1,1
1,-0.385439,0.0,0.0,1.204,-0.02376,-0.832936,-0.742471,0.560453,0.276620,1.056808,-1.486171,1.564269,-0.517023,25,0,4,26,0,1,0
2,0.460409,0.0,0.0,1.204,-0.02376,-0.135674,-0.358856,0.933117,-0.788063,1.056808,-0.546976,1.549752,-0.509288,25,0,4,26,0,1,0


In [ ]:
# Partition data
y = ml_kick['specialTeamsResult']
X = ml_kick.drop(['specialTeamsResult'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# KNN clasification
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, knn_pred))

Accuracy: 0.9739212897107634


In [ ]:
# RandomForest classification
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

Accuracy: 0.9981033665244191


# Punt

In [ ]:
joined_df['time']

0       18:14.0
1       40:25.7
2       46:32.6
3       22:13.7
4       26:14.6
         ...   
1861    17:03.8
1862    23:23.3
1863    41:22.6
1864    44:16.1
1865    55:43.9
Name: time, Length: 1866, dtype: object

* Prepar for fitting

In [ ]:
# Drop records with too many null
# 'returnerId'  '44029;46130'
ml_punt = joined_df.drop(['kickerId','Column_1.1','returnerId','level_0','index','Column_1','gameId','playId','kickBlockerId','penaltyCodes','penaltyJerseyNumbers','penaltyYards','passResult','kickReturnYardage','gameClock','frameId','gameId.1','playId.1'], axis=1)
ml_punt = ml_punt.drop(['s','a','dis','o','dir','displayName','jerseyNumber','position'], axis=1)
# Fill with '' since the dtype of column is object
ml_punt['yardlineSide'] = ml_punt['yardlineSide'].fillna('')
ml_punt['kickLength'] = ml_punt['kickLength'].fillna(0)
# Transform time format and convert to total seconds
ml_punt['time'] = pd.to_datetime(ml_punt['time'], format = '%M:%S.%f').dt.time
idx = pd.to_timedelta(ml_punt['time'].astype(str))
ml_punt['time'] = idx.dt.total_seconds()
ml_punt.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1866 entries, 0 to 1865
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   quarter                 1866 non-null   int64  
 1   down                    1866 non-null   int64  
 2   yardsToGo               1866 non-null   int64  
 3   possessionTeam          1866 non-null   object 
 4   specialTeamsPlayType    1866 non-null   object 
 5   specialTeamsResult      1866 non-null   object 
 6   yardlineSide            1866 non-null   object 
 7   yardlineNumber          1866 non-null   int64  
 8   preSnapHomeScore        1866 non-null   int64  
 9   preSnapVisitorScore     1866 non-null   int64  
 10  kickLength              1866 non-null   float64
 11  playResult              1866 non-null   int64  
 12  absoluteYardlineNumber  1866 non-null   int64  
 13  time                    1866 non-null   float64
 14  x                       1866 non-null   

In [ ]:
# Use label encoder to transform categorical data
subset = ml_punt.loc[:,['possessionTeam','specialTeamsPlayType','specialTeamsResult','yardlineSide','event','team','playDirection']]
subset = subset.apply(LabelEncoder().fit_transform)
subset.head(3)

,possessionTeam,specialTeamsPlayType,specialTeamsResult,yardlineSide,event,team,playDirection
0,25,0,5,26,0,1,1
1,25,0,6,26,0,1,1
2,1,0,2,2,0,0,1


In [ ]:
ml_punt = ml_punt.drop(['possessionTeam','specialTeamsPlayType','specialTeamsResult','yardlineSide','event','team','playDirection'], axis=1)
norm = StandardScaler()
ml_punt = pd.DataFrame(norm.fit_transform(ml_punt),columns = ml_punt.columns)
ml_punt

,quarter,down,yardsToGo,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,kickLength,playResult,absoluteYardlineNumber,time,x,y,nflId
0,-1.235080,0.0,-0.876376,-2.085899,-1.033126,-1.007635,1.098812,-0.379963,-1.929040,-0.607440,-1.631561,1.565872,0.924634
1,-1.235080,0.0,1.500540,0.251343,-1.033126,-0.673408,2.019596,0.350976,-0.660620,0.696261,-0.824320,1.602326,0.924634
2,-0.324867,0.0,1.670320,-1.826205,-1.033126,-0.673408,0.382646,0.675838,-1.788104,1.055446,-1.546808,0.942515,-0.614423
3,-0.324867,0.0,-0.197257,-0.094915,-0.735792,-0.339182,1.303431,1.406777,0.936650,-0.372779,1.009801,0.723794,0.924634
4,-0.324867,0.0,-0.197257,-0.527737,-0.735792,-0.339182,1.201121,1.650423,-1.083426,-0.136944,-1.099468,-1.273864,-0.614423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,0.585346,0.0,1.500540,-1.479947,0.354431,0.440680,2.019596,0.432191,1.688306,-0.676164,1.516527,-1.007753,1.038180
1862,0.585346,0.0,0.651641,1.376682,0.354431,0.440680,0.075718,0.432191,0.138015,-0.304643,-0.334903,1.004486,-2.114708
1863,0.585346,0.0,1.160980,0.511037,1.048210,0.440680,1.712668,0.107330,-0.519684,0.751964,-0.738971,0.450392,-2.114708
1864,0.585346,0.0,-0.706597,-0.614302,1.048210,0.440680,-4.630515,-5.334104,1.218521,0.921816,1.208254,-0.985881,1.038180


In [ ]:
# Concat numerical and encoded categorical data
ml_punt = pd.concat([ml_punt, subset], axis = 1)
ml_punt.head(3)

,quarter,down,yardsToGo,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,kickLength,playResult,absoluteYardlineNumber,time,x,y,nflId,possessionTeam,specialTeamsPlayType,specialTeamsResult,yardlineSide,event,team,playDirection
0,-1.235080,0.0,-0.876376,-2.085899,-1.033126,-1.007635,1.098812,-0.379963,-1.929040,-0.607440,-1.631561,1.565872,0.924634,25,0,5,26,0,1,1
1,-1.235080,0.0,1.500540,0.251343,-1.033126,-0.673408,2.019596,0.350976,-0.660620,0.696261,-0.824320,1.602326,0.924634,25,0,6,26,0,1,1
2,-0.324867,0.0,1.670320,-1.826205,-1.033126,-0.673408,0.382646,0.675838,-1.788104,1.055446,-1.546808,0.942515,-0.614423,1,0,2,2,0,0,1


In [ ]:
# Partition data
y = ml_punt['specialTeamsResult']
X = ml_punt.drop(['specialTeamsResult'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# KNN clasification
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, knn_pred))

Accuracy: 0.375


In [ ]:
# RandomForest classification
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

Accuracy: 0.5892857142857143


# Field Goal + Extra Point

In [ ]:
# Drop records with too many null
# 'returnerId'  '44029;46130'
ml_FE = joined_df.drop(['kickerId','gameId','playId','returnerId','level_0','index','Column_1','kickBlockerId','penaltyCodes','penaltyJerseyNumbers','penaltyYards','passResult','kickReturnYardage','time','frameId','gameId.1','playId.1','Column_1.1'], axis=1)
ml_FE = ml_FE.drop(['s','a','dis','o','dir','displayName','jerseyNumber','position'], axis=1)
# Fill with '' since the dtype of column is object
ml_FE['yardlineSide'] = ml_FE['yardlineSide'].fillna('')
ml_FE['kickLength'] = ml_FE['kickLength'].fillna(0)
# Transform time format and convert to total seconds
ml_FE['gameClock'] = pd.to_datetime(ml_FE['gameClock'], format = '%M:%S:%f').dt.time
idx = pd.to_timedelta(ml_FE['gameClock'].astype(str))
ml_FE['gameClock'] = idx.dt.total_seconds()
ml_FE.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5981 entries, 0 to 5980
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   quarter                 5981 non-null   int64  
 1   down                    5981 non-null   int64  
 2   yardsToGo               5981 non-null   int64  
 3   possessionTeam          5981 non-null   object 
 4   specialTeamsPlayType    5981 non-null   object 
 5   specialTeamsResult      5981 non-null   object 
 6   yardlineSide            5981 non-null   object 
 7   yardlineNumber          5981 non-null   int64  
 8   gameClock               5981 non-null   float64
 9   preSnapHomeScore        5981 non-null   int64  
 10  preSnapVisitorScore     5981 non-null   int64  
 11  kickLength              5981 non-null   float64
 12  playResult              5981 non-null   int64  
 13  absoluteYardlineNumber  5981 non-null   int64  
 14  x                       5981 non-null   

In [ ]:
# Use label encoder to transform categorical data
subset = ml_FE.loc[:,['possessionTeam','specialTeamsPlayType','specialTeamsResult','yardlineSide','event','team','playDirection']]
subset = subset.apply(LabelEncoder().fit_transform)
subset.head(3)

,possessionTeam,specialTeamsPlayType,specialTeamsResult,yardlineSide,event,team,playDirection
0,1,1,1,25,0,0,0
1,25,1,1,1,0,1,0
2,1,1,1,25,0,0,1


In [ ]:
ml_FE = ml_FE.drop(['possessionTeam','specialTeamsPlayType','specialTeamsResult','yardlineSide','event','team','playDirection'], axis=1)
norm = StandardScaler()
ml_FE = pd.DataFrame(norm.fit_transform(ml_FE),columns = ml_FE.columns)
ml_FE

,quarter,down,yardsToGo,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,kickLength,playResult,absoluteYardlineNumber,x,y,nflId
0,-1.414076,1.178398,-0.040187,-1.895517,-0.279485,-1.304627,-1.409255,0.239537,0.281798,-1.526635,-1.597108,-2.220022,-3.314364
1,-0.472985,1.178398,1.019604,-1.209369,-0.012002,-1.304627,-1.120533,0.493448,0.281798,-1.378214,-1.386668,-2.068721,1.442755
2,-0.472985,1.178398,2.927230,2.358601,-0.938500,-1.059876,-1.120533,1.813788,0.281798,0.343470,0.069573,0.438548,-3.314364
3,0.468107,-0.869397,-0.676063,-0.248762,-0.380275,-0.570375,-0.831811,-0.826891,0.281798,-1.170424,-1.094057,-0.858315,1.442755
4,1.409198,-0.869397,-0.676063,-0.248762,0.825334,-0.488792,-0.254367,-0.826891,0.281798,-1.170424,-1.104078,-0.876327,-3.314364
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5976,-0.472985,-0.869397,-0.676063,-0.248762,-0.120546,-0.162458,-0.543089,-0.826891,0.281798,-1.170424,-1.112095,-2.065119,1.461650
5977,0.468107,-0.869397,-0.676063,-0.248762,-0.391905,0.327043,-0.158127,-0.826891,0.281798,-1.170424,-1.112496,0.543018,-3.784088
5978,1.409198,1.178398,2.079396,0.986305,0.038393,0.408627,0.611798,1.305965,0.281798,0.640312,0.482034,-0.786267,-3.784088
5979,1.409198,1.178398,1.867438,2.770290,-1.012154,0.653377,0.611798,1.966135,0.281798,-0.517372,-0.225042,-2.083131,1.461650


In [ ]:
# Concat numerical and encoded categorical data
ml_FE = pd.concat([ml_FE, subset], axis = 1)
ml_FE.head(3)

,quarter,down,yardsToGo,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,kickLength,playResult,absoluteYardlineNumber,x,y,nflId,possessionTeam,specialTeamsPlayType,specialTeamsResult,yardlineSide,event,team,playDirection
0,-1.414076,1.178398,-0.040187,-1.895517,-0.279485,-1.304627,-1.409255,0.239537,0.281798,-1.526635,-1.597108,-2.220022,-3.314364,1,1,1,25,0,0,0
1,-0.472985,1.178398,1.019604,-1.209369,-0.012002,-1.304627,-1.120533,0.493448,0.281798,-1.378214,-1.386668,-2.068721,1.442755,25,1,1,1,0,1,0
2,-0.472985,1.178398,2.927230,2.358601,-0.938500,-1.059876,-1.120533,1.813788,0.281798,0.343470,0.069573,0.438548,-3.314364,1,1,1,25,0,0,1


In [ ]:
# Partition data
y = ml_FE['specialTeamsResult']
X = ml_FE.drop(['specialTeamsResult'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# KNN clasification
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, knn_pred))

Accuracy: 0.9793871866295265


In [ ]:
# RandomForest classification
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

Accuracy: 0.9949860724233983
